# Example II

This experiment how to model coexisting phenomena

In [ ]:
import matplotlib.pyplot as plt
import network_diffusion as nd
import numpy as np
import pandas as pd

from tqdm import tqdm
from utils import functions, models, networks

%load_ext autoreload
%autoreload 2

In [ ]:
functions.set_seed(42)
plt.rcParams.update({'font.size': 12})

### Define parameters

In [ ]:
# for the model
ALPHA = 0.19
ALPHA_PRIME = 0.35 * 0.19
BETA = 0.10
GAMMA = 0.01
DELTA = GAMMA + 1 - 0.3
EPSILON = GAMMA

# for the simulation
N_EPOCHS = 60
PATIENCE = None
N_REPETITIONS = 100

### Load the network

In [ ]:
net, name = networks.get_aucs_network(file_path="aucs.mpx"), "sir_ua_aucs"
print(net)

### Load the model

In [ ]:
model = models.SIR_UAModel(
    alpha = ALPHA,
    beta = BETA,
    alpha_prime = ALPHA_PRIME,
    beta_prime = BETA,
    gamma = GAMMA,
    delta = DELTA,
    epsilon = EPSILON,
    ill_seeds = 5,
    aware_seeds = 5,
)

In [ ]:
print(model)

### Perform the simulation

In [ ]:
all_logs = []
for i in tqdm(range(N_REPETITIONS)):
    experiment = nd.Simulator(model, net.copy())
    run_logs = experiment.perform_propagation(n_epochs=N_EPOCHS, patience=PATIENCE)
    all_logs.append(run_logs._global_stats_converted)

### Convert the logs and plot the results

In [ ]:
mean_contagion = functions.get_mean_log([log["contagion"] for log in all_logs])
mean_awareness = functions.get_mean_log([log["awareness"] for log in all_logs])
mean_core = pd.DataFrame([mean_contagion["I"], mean_awareness["A"]]).T

std_contagion = functions.get_std_log([log["contagion"] for log in all_logs])
std_awareness = functions.get_std_log([log["awareness"] for log in all_logs])
std_core = pd.DataFrame([std_contagion["I"], std_awareness["A"]]).T

result = {"mean": mean_core, "std": std_core}

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)
fig.set_size_inches(10,5)
colors = plt.cm.jet(np.linspace(0, 1, len(result) + 1))
res_nb = 0

_i = result["mean"]["I"].to_numpy()
_i_std = result["std"]["I"].to_numpy()
_a = result["mean"]["A"].to_numpy()
_a_std = result["std"]["A"].to_numpy()
_x = np.arange(0, len(_i))

ax[0].plot(_x, _i, label=rf"Infected", color=colors[res_nb])
ax[0].fill_between(_x, _i-_i_std, _i+_i_std, alpha=0.1, color=colors[res_nb])
ax[1].plot(_x, _a, label=rf"Aware", color=colors[res_nb])
ax[1].fill_between(_x, _a-_a_std, _a+_a_std, alpha=0.1, color=colors[res_nb])

res_nb += 1

for a in ax:
    a.grid(True)
    a.set_xlabel("Epoch")
    a.set_xticks(np.arange(0, _x.max()+1, 5))
    a.set_ylabel("nb of Agents")

    ax[0].legend()
    ax[1].legend(loc="lower right")

fig.tight_layout()